In [3]:
!pip install torch==2.4.1

  Using cached torch-2.4.1-cp310-cp310-manylinux1_x86_64.whl.metadata (26 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64

In [6]:
!pip install torch_geometric


  Using cached torch_geometric-2.6.1-py3-none-any.whl.metadata (63 kB)
Using cached torch_geometric-2.6.1-py3-none-any.whl (1.1 MB)


In [1]:
import os

# Change to the desired directory
os.chdir('/home/ec2-user/SageMaker/SComGNN-main')

# Verify the current working directory
print(os.getcwd())

/home/ec2-user/SageMaker/SComGNN-main


In [2]:
from model import SComGNN
from utils import *
import random
import os
import argparse

In [87]:
class argC:
    def __init__(self):
        self.ckpt_path = 'None'
        self.device = 'cuda:0'
        self.mode = None
        self.embedding_dim = 16
        # self.dataset = 'Home_and_Kitchen'
        # self.dataset = 'Grocery_and_Gourmet_Food'
        # self.dataset = 'Electronics'
        # self.dataset = 'Toys_and_Games'
        self.dataset = 'Cell_Phones_and_Accessories'


In [88]:

args = argC()

In [89]:
print('Dataset: {}'.format(args.dataset), flush=True)
device = torch.device(args.device if torch.cuda.is_available() else "cpu")
# device = torch.device(args.device)

ckpt_path = "checkpoints/"
if not os.path.exists(ckpt_path):
    os.mkdir(ckpt_path)

if args.ckpt_path == 'None':
    ckpt_path = "checkpoints/{}".format(args.dataset)
else:
    ckpt_path = "checkpoints/{}".format(args.ckpt_path)
if not os.path.exists(ckpt_path):
    os.mkdir(ckpt_path)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


#load dataset
features, price_bin, com_edge_index, train_set, val_set, test_set = load_dataset(args.dataset)

num_items = features.shape[0]
adj = generate_adj(com_edge_index, num_items)

features = torch.FloatTensor(features).to(device)
price_bin = torch.LongTensor(price_bin).to(device)
adj = adj.to(device)
train_set = torch.LongTensor(train_set).to(device)
val_set = torch.LongTensor(val_set).to(device)
test_set = torch.LongTensor(test_set).to(device)

Dataset: Cell_Phones_and_Accessories
Finish generating adjacent matrix


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/preprocessing/_discretization.py:306: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  warnings.warn(


In [90]:
train_set.max()

tensor(1486, device='cuda:0')

In [91]:
val_set.shape

torch.Size([635, 102])

In [92]:
train_set.shape

torch.Size([5334, 4])

In [93]:
model = SComGNN(args.embedding_dim, 20, args.mode)
model = model.to(device)
model.load_state_dict(torch.load('{}/model_gat.pkl'.format(ckpt_path)))

/tmp/ipykernel_33612/42334272.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('{}/model_gat.pkl'.format(ckpt_path)))


<All keys matched successfully>

In [94]:





with torch.no_grad():
    model.eval()
    hr5_score, hr10_score, ndcg_score, scores, labels= model.inference(features, price_bin, adj, test_set)


In [95]:
labels.shape

torch.Size([635, 101])

In [96]:
labels[:, 0].sum()


tensor(635., device='cuda:0')

In [97]:
ranking = torch.argsort(scores, dim=1, descending=True)


In [98]:
with open('./data_preprocess/tmp/{}_id_dict.txt'.format(args.dataset), 'r') as file:
# Read the entire content of the file
    content = file.read()


lines = content.split('\n')

# Create an empty dictionary to store the key-value pairs
data_dict = {}

# Process each line
for line in lines[:-1]:
    # Split each line by the tab character
    key, value = line.split('\t')

    
    # Convert the value to an integer (assuming all values are integers)
    value = int(value)
    
    # Add the key-value pair to the dictionary
    data_dict[value] = key


In [99]:
ranking.shape

torch.Size([635, 101])

In [100]:
asin_set = test_set[:, 1:]

In [101]:
test_set_asin = np.array([
    [data_dict[int(x)] for x in row]  # Use 'Unknown' for missing keys
    for row in test_set
])

In [102]:
ordered_asin_set = torch.stack([asin_set[i, ranking[i]] for i in range(asin_set.size(0))])


In [103]:
test_set_asin[:, 0].shape

(635,)

In [104]:
mapped_ordered_asin_set = np.array([
    [data_dict[int(x)] for x in row]  # Use 'Unknown' for missing keys
    for row in ordered_asin_set
])


In [105]:
anchor_asin_column = test_set_asin[:, 0].reshape(-1, 1)

# Combine the asin_column with the existing matrix
anchor_ordered_asin = np.column_stack((anchor_asin_column, mapped_ordered_asin_set))

In [106]:

np.save('../llm_data/test_label_anchor_ordered_asin_{}_gat.npy'.format(args.dataset), test_set_asin[:, 1].reshape(-1, 1))

In [107]:
np.save('../llm_data/test_anchor_ordered_asin_{}_gat.npy'.format(args.dataset), anchor_ordered_asin)

In [13]:
meta_file = open('/home/ec2-user/SageMaker/DHGAN/DHGAN_code/data_preprocess/raw_data/meta_{}.json'.format(args.dataset)).readlines()


In [6]:
import json

def process_metadata(file_path):
    result = {}
    
    with open(file_path, 'r') as file:
        for line in file:
            item = json.loads(line)
            
            asin = item.get('asin')
            if asin:
                title = item.get('title', '')
                brand = item.get('brand', '')
                description = item.get('description', '')
                
                result[asin] = {
                    'title': title,
                    'brand': brand,
                    'description': description
                }
    
    return result

# Usage
file_path = '/home/ec2-user/SageMaker/SComGNN-main/data_preprocess/raw_data/meta_{}.json'.format(args.dataset)
# file_path = '/home/ec2-user/SageMaker/DHGAN/DHGAN_code/data_preprocess/raw_data/meta_{}.json'.format(args.dataset)
metadata_dict = process_metadata(file_path)


In [7]:
output_file_path =  '../llm_data/llm_meta_{}.json'.format(args.dataset)
with open(output_file_path, 'w') as outfile:
        json.dump(metadata_dict, outfile, indent=2)
    
